In [1]:
from bs4 import BeautifulSoup
from requests import get
import re
from datetime import datetime

In [2]:
APARTMENT = 'The Verge'

FLOORPLANS = [
    ('The Cusp', '17', 1020, 2, 2.0),
    ('Greenway', '18', 1134, 2, 2.0)
]

url = 'https://vergeapartments.com/floorplans/?action=check-availability&floorplan=18'
page_response = get(url, timeout=5, headers={'X-Requested-With': 'XMLHttpRequest'})
soup = BeautifulSoup(page_response.content, "html.parser")

print(soup.prettify())

<div class="floorplan-listing-b__row floorplan-listing-b__row--reserve" id="reserve-row" style="display:none;">
 <a class="floorplan-listing-b__availability-table-close" href="" id="close-reserve-row">
  ×
 </a>
 <p class="floorplan-listing-b__row-title">
  Available Apartments
 </p>
 <div class="check-availability" data-js-hook="panel" data-skip-step-init="0" data-type="check-availability">
  <table class="check-availability__table">
   <tr class="check-availability__row check-availability__row--head">
    <th class="check-availability__head check-availability__head--equal">
     Apt #
    </th>
    <th class="check-availability__head check-availability__head--equal">
     Starting At
    </th>
    <th class="check-availability__head check-availability__head--equal">
     Deposit
    </th>
    <th class="check-availability__head">
     Availability
    </th>
    <th class="check-availability__head">
     Lease
    </th>
   </tr>
   <tr class="check-availability__row">
    <td class="c

In [3]:
listings = []
for fp in FLOORPLANS:
    
    url = 'https://vergeapartments.com/floorplans/?action=check-availability&floorplan=' + fp[1]
    page_response = get(url, timeout=5, headers={'X-Requested-With': 'XMLHttpRequest'})
    soup = BeautifulSoup(page_response.content, "html.parser")
    
    units = soup.select('tr[class="check-availability__row"]')
    for unit in units:

        data = {}

        data['floorplan'] = fp[0]
        data['size'] = fp[2]
        data['beds'] = fp[3]
        data['baths'] = fp[4]

        # Unit Number
        unit_str = unit.find('span', text='Apt # ').parent.text.strip()
        data['unit'] = re.search(r'Apt # (\d+)', unit_str).group(1)

        # Floor (based on Unit #)
        data['floor'] = int(data['unit'][1]) if data['unit'][0] == '1' else int(data['unit'][1]) - 2

        # Available Date
        availability_str = unit.find('span', text='Availability ').parent.text
        if 'now' in availability_str.lower():
            data['available_dt'] = 'Available Now'
        else:
            # The original date is in this format: "April 17, 2019"
            r = r'Available on ([A-Za-z]+ \d{1,2}, \d{4})'
            date_str = re.search(r, availability_str).group(1)

            # The final date is in this format: "2019-04-17"
            data['available_dt'] = str(datetime.strptime(date_str, '%B %d, %Y').date())

        # Lease Terms
        data['terms'] = []
        terms = soup.find('tr', {'class': 'check-availability__row--lease-term', 'data-unit': data['unit']}).find_all('label')
        for term in terms:
            r = re.search('(\d{1,2}) Months months at \$(\d{4})\/mo', term.text)
            length = r.group(1)
            price = int(r.group(2))

            if int(length) > 5:
                data['terms'].append({'k': length, 'v': price})


        listings.append(data)


listings

[{'floorplan': 'The Cusp',
  'size': 1020,
  'beds': 2,
  'baths': 2.0,
  'unit': '1466',
  'floor': 4,
  'available_dt': '2019-04-17',
  'terms': [{'k': '15', 'v': 2428},
   {'k': '14', 'v': 2440},
   {'k': '13', 'v': 2428},
   {'k': '12', 'v': 2462},
   {'k': '11', 'v': 2483},
   {'k': '10', 'v': 2467},
   {'k': '9', 'v': 2557},
   {'k': '8', 'v': 2597},
   {'k': '7', 'v': 2649},
   {'k': '6', 'v': 2638}]},
 {'floorplan': 'The Cusp',
  'size': 1020,
  'beds': 2,
  'baths': 2.0,
  'unit': '2639',
  'floor': 4,
  'available_dt': '2019-04-21',
  'terms': [{'k': '15', 'v': 2363},
   {'k': '14', 'v': 2374},
   {'k': '13', 'v': 2363},
   {'k': '12', 'v': 2396},
   {'k': '11', 'v': 2417},
   {'k': '10', 'v': 2402},
   {'k': '9', 'v': 2490},
   {'k': '8', 'v': 2530},
   {'k': '7', 'v': 2582},
   {'k': '6', 'v': 2573}]},
 {'floorplan': 'Greenway',
  'size': 1134,
  'beds': 2,
  'baths': 2.0,
  'unit': '1367',
  'floor': 3,
  'available_dt': 'Available Now',
  'terms': [{'k': '15', 'v': 2515},